In [ ]:
import os
import numpy as np
import pandas as pd

# -------------------------------
# Configuration
# -------------------------------
BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(1, 150)]  # adjust subject range

ROIS = {
    'Frontal': ['Fp1','Fz','F3','F7','F4','F8','Fp2','AF7','AF3','AFz','F1','F5','F6','AF8','AF4','F2','FCz'],
    'Frontocentral': ['FT9','FC5','FC1','FT7','FC3','FC6','FC2','FT8'],
    'Central': ['C3','C1','C5','Cz','C4','C6','C2'],
    'Temporal': ['T7','TP9','TP7','T8','TP10','TP8'],
    'Parietal': ['CP5','CP1','CP3','CPz','CP6','CP2','P3','P7','P1','P5','P4','P8','P6','P2'],
    'Occipital': ['O1','Oz','O2','PO7','PO3','POz','PO4','PO8']
}

BANDS = ['delta','theta','alpha','beta']

# -------------------------------
# Loop over subjects
# -------------------------------
for SUBJ in SUBJECTS:
    DATA_DIR = os.path.join(BASE_DIR, SUBJ, "data")
    band_file = os.path.join(DATA_DIR, f"{SUBJ}_DWT_band_energy.csv")  # generated by your DWT script
    
    if not os.path.exists(band_file):
        print(f"Missing DWT file for {SUBJ}, skipping...")
        continue

    df = pd.read_csv(band_file)
    roi_rows = []

    for roi, channels in ROIS.items():
        roi_df = df[df['channel'].isin(channels)]
        
        if len(roi_df) == 0:
            print(f"  ROI '{roi}' has no channels for {SUBJ}, skipping...")
            continue
        
        # Track missing channels
        missing_ch = set(channels) - set(roi_df['channel'])
        if missing_ch:
            print(f"  ROI '{roi}' missing channels for {SUBJ}: {missing_ch}")

        row = {'subject': SUBJ, 'roi': roi}
        
        # Mean power per band
        for band in BANDS:
            row[f'{band}_power'] = roi_df[f'{band}_energy'].mean()
        
        # Cognitive PD ratios
        row['theta_alpha'] = row['theta_power'] / (row['alpha_power'] + 1e-12)
        row['theta_beta'] = row['theta_power'] / (row['beta_power'] + 1e-12)
        row['ab_delta'] = (row['alpha_power'] + row['beta_power']) / (row['delta_power'] + 1e-12)
        row['alpha_delta'] = row['alpha_power'] / (row['delta_power'] + 1e-12)

        roi_rows.append(row)

    # Save ROI-level features
    if roi_rows:
        df_roi = pd.DataFrame(roi_rows)
        out_file = os.path.join(DATA_DIR, f"{SUBJ}_ROI_band_ratios.csv")
        df_roi.to_csv(out_file, index=False)
        print(f"Saved ROI features → {out_file}")
#  ROI 'Frontocentral' missing channels for sub-004: {'FT9'}
#  ROI 'Occipital' missing channels for sub-004: {'PO3', 'PO4'}


  ROI 'Frontocentral' missing channels for sub-001: {'FT9'}
  ROI 'Occipital' missing channels for sub-001: {'PO3', 'PO4'}
Saved ROI features → C:\Users\User\Documents\EEG_Project\rEEG\sub-001\data\sub-001_ROI_band_ratios.csv
  ROI 'Frontocentral' missing channels for sub-002: {'FT9'}
  ROI 'Occipital' missing channels for sub-002: {'PO3', 'PO4'}
Saved ROI features → C:\Users\User\Documents\EEG_Project\rEEG\sub-002\data\sub-002_ROI_band_ratios.csv
  ROI 'Frontocentral' missing channels for sub-003: {'FT9'}
  ROI 'Occipital' missing channels for sub-003: {'PO3', 'PO4'}
Saved ROI features → C:\Users\User\Documents\EEG_Project\rEEG\sub-003\data\sub-003_ROI_band_ratios.csv
  ROI 'Frontocentral' missing channels for sub-004: {'FT9'}
  ROI 'Occipital' missing channels for sub-004: {'PO3', 'PO4'}
Saved ROI features → C:\Users\User\Documents\EEG_Project\rEEG\sub-004\data\sub-004_ROI_band_ratios.csv
  ROI 'Frontocentral' missing channels for sub-005: {'FT9'}
  ROI 'Occipital' missing channels f